# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### Tecnológico de Monterrey

### Prof Dra. Grettel Barceló Alonso y Dr. Luis Eduardo Falcón Morales

## Avance III de Proyecto

## Modelo Base

## Integrantes del Equipo:
### - Erika Cardona Rojas            A01749170
### - Miriam Bönsch                  A01330346
### - Mardonio Manuel Román Ramírez  A01795265

In [12]:
# Librerias
import pandas as pd
import yaml
import numpy as np

# Cargando Yaml
with open("../config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

# Importando modulos
import sys
from pathlib import Path

parent_folder = str(Path.cwd().parent)
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

from src import filtering_vars as fv

In [2]:
# Cargando Base de Datos
df = pd.read_csv(r"../Data/DF_Final_No_Filtered.csv")

In [3]:
df.shape

(168, 540)

# Filtrado De Variables

Dada nuestra previa entrega, se ha discutido con los expertos la viabilidad de eliminación de las variables propuestas.

A continuación se aplicará la eliminación correspondiente a su retroalimentación.

In [4]:
# Creando PAD_media a partir de PAD_izq y PAD_der
df['PAD_media'] = df[['PAD_izq', 'PAD_der']].mean(axis=1)

In [5]:
# Eliminando variables correlacionadas y con un varianza casi nula.

vars_delete = config['Filtrado_Variables']['Umbral_Varianza'] + config['Filtrado_Variables']['Correlacionadas']

vars_delete = list(set(vars_delete))

df = df.drop(vars_delete, axis='columns')
print(f"Dada nuestra conversación con nuestros compañeros de Barcelona, se eliminarán {len(vars_delete)} variables.")

Dada nuestra conversación con nuestros compañeros de Barcelona, se eliminarán 86 variables.


# Reestructura De Datos

Los datos se encuentran repitiendo a cada sujeto 4 veces. Pues cada registro se divide en `Treatment` (Intervención y Control) y `Time` (Pre y Post).

Nos interesa conocer los efectos del experimento en el periodo Post de la Intervención con tomate.

Por lo que se estructurará la base de datos, de manera que nos funcione así.

En este modelo, predeciremos el efecto en el tratamiento (delta bdnf), así mismo aprovecharemos la manipulación de los datos para generar otra variable objetivo, que sería el
efecto neto (delta BDNF Intervención - delta BDNF Control).

In [6]:
# Generando deltas.

# Pivoteando para tener pre y post como columnas.
df_deltas = df.pivot(index=['id','Treatment'], columns='Time', values='bdnf').reset_index()
# Calculando delta por tratamiento
df_deltas['delta_bdnf'] = df_deltas['Post'] - df_deltas['Pre']

In [7]:
# Primer variable: Delta en Periodo de Tratamiento.
df_delta_I = df_deltas[df_deltas['Treatment'] == 'Intervencion']
df_delta_I = df_delta_I[['id','delta_bdnf']].rename(columns={'delta_bdnf':'delta_bdnf_Int'})

In [8]:
# Generando efecto neto.
df_delta_N = df_deltas.pivot(index=['id'], columns='Treatment', values='delta_bdnf').reset_index()
# Calculando delta por tratamiento
df_delta_N['delta_bdnf_Neto'] = df_delta_N['Intervencion'] - df_delta_N['Control']

df_delta_N = df_delta_N[['id','delta_bdnf_Neto']]

deltas = df_delta_N.merge(df_delta_I)

del df_delta_N,df_delta_I,df_deltas

Generando Dataset para predecir con el estado basal del periodo intervención.

Se agregarán variables objetivos: 

    - delta bdnf en periodo de intervención
    - delta bdnf neto : delta intervención - delta control

In [ ]:
# Generando Dataset.
df_pred = df[(df['Treatment'] == 'Intervencion') & (df['Time'] == 'Pre')].reset_index(drop=True)

# Agregando Variables Objetivo
df_pred = df_pred.merge(deltas)
# Eliminando columnas no deseadas en entrenamiento
df_pred = df_pred.drop(columns= config['Tratamiento_Variables']['Objetivos'] + ['id', 'grupo'])

# Para este modelo, eliminaremos la del efecto neto, únicamente modelaremos el efecto en intervención
del df_pred['delta_bdnf_Neto']

# Modelado

In [13]:
# Infiriendo tipo de variables
type_cols = fv.infer_var_types(df_pred,df_pred.columns)

In [14]:
type_cols

{'frec_alcohol_audit': 'Ordinal',
 'puntaje_audit': 'Ordinal',
 'Palabras_moca': 'Numerica',
 'memoria_moca': 'Ordinal',
 'puntaje_moca': 'Ordinal',
 'edad': 'Numerica',
 'edo_civil': 'Ordinal',
 'personas_hogar': 'Ordinal',
 'nivel_e': 'Ordinal',
 'ejercicio': 'Binaria',
 'af_trabajo': 'Ordinal',
 'af_libre': 'Ordinal',
 'cafe': 'Ordinal',
 'tipo_dieta': 'Ordinal',
 'tabaco': 'Ordinal',
 'alcohol': 'Binaria',
 'tipo_alcohol': 'Ordinal',
 'frec_alcohol': 'Ordinal',
 'alergias_alim': 'Binaria',
 'horas_sueño': 'Numerica',
 'horas_sueño_fs': 'Numerica',
 'descansa': 'Binaria',
 'siesta': 'Ordinal',
 'cambios_apetito': 'Binaria',
 'cambios_sueño': 'Binaria',
 'int_quirugica': 'Binaria',
 'covid': 'Ordinal',
 'fam_cardiacos': 'Binaria',
 'fam_acvascular': 'Binaria',
 'fam_obesidad': 'Binaria',
 'fam_diabetes': 'Binaria',
 'fam_colesterol': 'Binaria',
 'fam_hta': 'Binaria',
 'fam_enfgastro': 'Binaria',
 'otras_med': 'Binaria',
 'vitaminas': 'Binaria',
 'suplementos': 'Binaria',
 'prebiotico